In [2]:
! pip install py2neo
from py2neo import Graph
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import numpy as np
#i-0e23d19f0d8795714

    100% |████████████████████████████████| 71kB 34.0MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 37.1MB/s ta 0:00:01
    100% |████████████████████████████████| 184kB 49.6MB/s ta 0:00:01
    100% |████████████████████████████████| 348kB 39.6MB/s ta 0:00:01
    100% |████████████████████████████████| 849kB 31.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for py2neo ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/86/7d/e3/35bd456d80e796ba1f98730b629352ebab5294c942e8fdd826
  Running setup.py bdist_wheel for neobolt ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/42/54/00/270de525f8f966d864bd3a0224d38dca2573b53c1a54267720
  Running setup.py bdist_wheel for neotime ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/d4/b4/c4/ab867cd0ac9e667f8cd269f9364fd1cb8ee75fe50f5ce97c79
Successfully built py2neo neobolt neotime
jupyter-console 5.2.0 has requirement prompt_toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.10 w

In [45]:
graph = Graph("bolt://3.220.233.169:7687", auth=("neo4j", "i-0e23d19f0d8795714"))

In [4]:
graph.run("CALL db.schema()").data()

[{'nodes': [(_-2:Category {constraints: [], indexes: ['name'], name: 'Category'}),
   (_-3:User {constraints: ['CONSTRAINT ON ( user:User ) ASSERT user.id IS UNIQUE'], indexes: [], name: 'User'}),
   (_-4:Review {constraints: ['CONSTRAINT ON ( review:Review ) ASSERT review.id IS UNIQUE'], indexes: [], name: 'Review'}),
   (_-1:Business {constraints: ['CONSTRAINT ON ( business:Business ) ASSERT business.id IS UNIQUE'], indexes: [], name: 'Business'})],
  'relationships': [(Review)-[:REVIEWS {}]->(Business),
   (User)-[:WROTE {}]->(Review),
   (User)-[:FRIEND {}]->(User),
   (Business)-[:IN_CATEGORY {}]->(Category)]}]

In [62]:
cats=graph.run("""MATCH (c:Category)
RETURN distinct c.id 
""").to_data_frame()

In [63]:
cats.shape

(1300, 1)

2336

In [58]:
# sns.set_style("whitegrid")
# plt.figure(figsize=(14,10))
# sns.boxplot(cat_count['count'])

In [59]:
# sum(cat_count['count'].loc[cat_count['count']==1])

In [60]:
# thi is a cypher query to find nodes (in this case categories) of a certain degree
# MATCH (c:Category)
# WITH c, apoc.node.degree(c,"IN_CATEGORY") as degree
# WHERE degree <5
# RETURN c.id,degree

In [64]:
user_count=graph.run("""MATCH (c:User)
return ID(c) as ident , apoc.node.degree(c, 'WROTE>') as count
""").to_data_frame()

In [65]:
user_count['count']

,count,ident
count,1.637138e+06,1.637138e+06
mean,4.083895e+00,3.490227e+06
std,1.469266e+01,2.408194e+06
min,1.000000e+00,1.950860e+05
25%,1.000000e+00,1.344378e+06
50%,1.000000e+00,3.063662e+06
75%,3.000000e+00,5.402947e+06
max,4.129000e+03,8.512231e+06


In [74]:
np.percentile(user_count['count'],[52,53])

array([1., 2.])

In [6]:
non_writers=user_count.loc[user_count['count']==0]

In [7]:
non_writers.shape

(486400, 2)

In [8]:
# non_writers=graph.run("""CALL apoc.periodic.iterate(
# "MATCH (u:User) WITH apoc.node.degree(u, 'WROTE>') as revs, u as u WHERE revs<1 RETURN u",
# "DETACH DELETE u", {batchSize:100, iterateList:true, parallel:false})""")

In [16]:
non_writers.data()

[{'batches': 4864,
  'total': 486400,
  'timeTaken': 244,
  'committedOperations': 486400,
  'failedOperations': 0,
  'failedBatches': 0,
  'retries': 0,
  'errorMessages': {},
  'batch': {'total': 4864, 'committed': 4864, 'failed': 0, 'errors': {}},
  'operations': {'total': 486400,
   'committed': 486400,
   'failed': 0,
   'errors': {}},
  'wasTerminated': False}]

constraints_added: 0
constraints_removed: 0
contained_updates: False
indexes_added: 0
indexes_removed: 0
labels_added: 0
labels_removed: 0
nodes_created: 0
nodes_deleted: 0
properties_set: 0
relationships_created: 0
relationships_deleted: 0

In [78]:
friend_count=graph.run("""MATCH (c:User)
RETURN ID(c) as ident, apoc.node.degree(c, 'FRIEND') as count
""").to_data_frame()

In [79]:
friend_count.describe()

,count,ident
count,1.637138e+06,1.637138e+06
mean,2.754502e-01,3.490227e+06
std,1.131968e+00,2.408194e+06
min,0.000000e+00,1.950860e+05
25%,0.000000e+00,1.344378e+06
50%,0.000000e+00,3.063662e+06
75%,0.000000e+00,5.402947e+06
max,3.450000e+02,8.512231e+06


In [108]:
lame_os1=graph.run("""
MATCH (b:User)
WITH ID(b) as ident, apoc.node.degree(b,"WROTE") as rdegree, apoc.node.degree(b,"FRIEND") as fdegree
WHERE rdegree<2 and fdegree<1
RETURN ident,rdegree,fdegree""").to_data_frame()

In [109]:
lame_os1.shape

(744673, 3)

In [110]:
lame_os1.head()

,fdegree,ident,rdegree
0,0,195091,1
1,0,195097,1
2,0,195109,1
3,0,195113,1
4,0,195115,1


In [111]:
lame_os1.describe()

,fdegree,ident,rdegree
count,744673.0,7.446730e+05,744673.0
mean,0.0,4.590623e+06,1.0
std,0.0,2.354850e+06,0.0
min,0.0,1.950910e+05,1.0
25%,0.0,2.618982e+06,1.0
50%,0.0,4.655676e+06,1.0
75%,0.0,6.635882e+06,1.0
max,0.0,8.512231e+06,1.0


In [113]:
lame_os=graph.run("""MATCH (b:User)
WITH ID(b) as ident, apoc.node.degree(b) as degree
WHERE degree<2
RETURN ident,degree""").to_data_frame()

In [114]:
lame_os.shape

(744673, 2)

In [103]:
lame_os.describe()

,ident,rdegree
count,1.637138e+06,1.637138e+06
mean,3.490227e+06,4.083895e+00
std,2.408194e+06,1.469266e+01
min,1.950860e+05,1.000000e+00
25%,1.344378e+06,1.000000e+00
50%,3.063662e+06,1.000000e+00
75%,5.402947e+06,3.000000e+00
max,8.512231e+06,4.129000e+03


In [107]:
lame_os.head()

,ident,rdegree
0,195091,1
1,195097,1
2,195099,1
3,195109,1
4,195113,1


In [17]:
biz_count=graph.run("""MATCH (c:Business)
RETURN ID(c) as ident , apoc.node.degree(c, '<REVIEWS') as count
""").to_data_frame()

In [18]:
biz_count.describe()

,count,ident
count,192609.000000,192609.000000
mean,34.712293,98522.725465
std,113.601101,55891.470567
min,0.000000,0.000000
25%,4.000000,50382.000000
50%,9.000000,98667.000000
75%,26.000000,146894.000000
max,8570.000000,195085.000000


In [103]:
# biz_select=graph.run("""
# MATCH (b:Business)
# WITH ID(b) as ident, apoc.node.degree(b,"<REVIEWS") as degree
# WHERE degree <5
# RETURN ident,degree""").to_data_frame()

In [57]:
# merge_categories=graph.run("""MATCH (n:Category)
# WITH trim(n.id) as name, collect(n) as nodes
# CALL apoc.refactor.mergeNodes(nodes) yield node
# RETURN node""")

In [61]:
merge_categories.to_data_frame()

,node
0,{'id': ' Golf'}
1,{'id': 'Active Life'}
2,{'id': ' Specialty Food'}
3,{'id': 'Restaurants'}
4,{'id': 'Dim Sum'}
5,{'id': 'Imported Food'}
6,{'id': 'Food'}
7,{'id': 'Chinese'}
8,{'id': 'Ethnic Food'}
9,{'id': 'Seafood'}


In [5]:
biz_select=graph.run("""
MATCH (u:User)
RETURN ID(u) as ident, u.useful, u.funny, u.cool""").to_data_frame()

In [6]:
biz_select.shape

(1637138, 4)

In [17]:
dftemp=biz_select.fillna(0)

In [48]:
dftemp.describe()

,ident,u.cool,u.funny,u.useful
count,1.637138e+06,1.637138e+06,1.637138e+06,1.637138e+06
mean,3.490227e+06,2.188046e+01,1.791785e+01,4.052770e+01
std,2.408194e+06,4.019675e+02,3.303504e+02,4.702421e+02
min,1.950860e+05,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.344378e+06,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.063662e+06,0.000000e+00,0.000000e+00,3.000000e+00
75%,5.402947e+06,3.000000e+00,3.000000e+00,1.300000e+01
max,8.512231e+06,1.486580e+05,1.302070e+05,1.542020e+05


In [25]:
dftemp.quantile(q=[.2,.35,.5,.65,.8,.95])

,ident,u.cool,u.funny,u.useful
0.20,1072521.40,0.0,0.0,0.0
0.35,1968091.95,0.0,0.0,1.0
0.50,3063662.50,0.0,0.0,3.0
0.65,4389233.05,1.0,1.0,7.0
0.80,5984803.60,5.0,4.0,19.0
0.95,7830374.15,36.0,33.0,110.0


going to bin the useful, funny and coll user ratings into these in order to generate nodes to quickly lookup these attributes.

| qunatile | ident | u.cool | u.funny | u.useful |
| --- | --- | --- | --- | --- |
| 0.20 | 1072521.40	| 0.0 | 0.0 | 0.0 |
| 0.35 | 1968091.95 | 0.0	 |0.0	|1.0 |
| 0.50 |	3063662.50	| 0.0	 | 0.0	| 3.0 | 
| 0.65 |	4389233.05	| 1.0	 | 1.0	| 7.0 |
| 0.80 |	5984803.60	| 5.0	 | 4.0	| 19.0 |
| 0.95 |	7830374.15	| 36.0	| 33.0	| 110.0 |


so cool and funny will get 4 bins and usefull will get 6.

In [30]:
# cool1=graph.run("""
# MATCH (n:Reputation {cool:1})
# MATCH (u:User) WHERE NOT exists(u.cool)
# RETURN u.name, u.co
# """).to_data_frame()

In [31]:
cool1

,n.cool,u.cool,u.name
0,1,None,Stefan
1,1,None,Simon
2,1,None,Reilly
3,1,None,A
4,1,None,Beer
5,1,None,Clint
6,1,None,Pamela
7,1,None,Rosy
8,1,None,Payam
9,1,None,Laura


In [33]:
# cool2=graph.run("""
# MATCH (n)
# WHERE ID(n)=6610
# MATCH (u:User) 
# WHERE 1<u.cool<5
# CREATE (u)-[:COOL_LEVEL]->(n)
# """)

ClientError: SyntaxError: Invalid input '1': expected whitespace, comment, a relationship pattern, ',', USING, WHERE, FROM GRAPH, CONSTRUCT, LOAD CSV, START, MATCH, UNWIND, MERGE, CREATE UNIQUE, CREATE, SET, DELETE, REMOVE, FOREACH, WITH, CALL, RETURN, UNION, ';' or end of input (line 3, column 16 (offset: 26))
"MATCH (u:User) 1<u.cool<5"
                 ^

In [29]:
cool2.to_data_frame()

,n.cool,u.cool,u.id
0,2,3,86J5DwcFk4f4In1Vxe2TvA
1,2,4,3CJUJILq7CLHk_9OrvpvQg
2,2,4,8NwU4TRsD3S6gIfBqFzDMQ
3,2,2,1fi6x4tnJtlVWaJmoIO9XA
4,2,4,_o740mSNRhMNYuPjSJoPLg
5,2,2,tAcB1ZohRtjozsymvqqHzQ
6,2,4,zFYFuufYWQSPj0r5lrKQKg
7,2,4,SvMGr_Oih7ivtzmnCL-Tmg
8,2,2,0nTJWagRU8_-KS1x7Zw5Yw
9,2,2,hzny0aF2jcUaY8rXN2bfJQ


In [36]:
cool_level_2=graph.run("""
MATCH (n)
WHERE ID(n)=6621
MATCH (u:User) WHERE u.cool=1
CREATE (u)-[:COOL_LEVEL]->(n)
""")

In [46]:
cool_level_1=graph.run("""
MATCH (n)
WHERE ID(n)=6557
MATCH (u:User) WHERE NOT exists(u.cool)
CREATE (u)-[:COOL_LEVEL]->(n)
""")

In [47]:
cool_level_1=graph.run("""
MATCH (n)
WHERE ID(n)=6625
MATCH (u:User) WHERE u.cool>4
CREATE (u)-[:COOL_LEVEL]->(n)
""")